In [1]:
import os
os.chdir('..')
%pwd

'/home/lucky/Chicken-Disease-Classification'

In [2]:
import tensorflow as tf

2023-08-26 18:57:16.723825: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-26 18:57:16.750683: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-26 18:57:16.751631: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-26 18:57:17.295878: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
model= tf.keras.models.load_model("artifacts/training/model.h5")

2023-08-26 18:57:18.179432: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-26 18:57:18.207397: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
from dataclasses import dataclass 
from pathlib import  Path


@dataclass(frozen= True)
class EvaluationConfig:
    path: Path
    training_data: Path 
    all_parameters: Path
    params_image_size: list
    params_batch_size: int

In [5]:
from ChickenDiseaseClassifier.constants import  *
from ChickenDiseaseClassifier.utils.common import  read_yaml, create_directories, save_json
from ChickenDiseaseClassifier.entity.config_entity import *

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path= CONFIG_FILE_PATH,
            params_file_path= PARAMS_FILE_PATH
            ):


        print(f"Loading configuration from {config_file_path}")
        print(f"Loading parameters from {params_file_path}")

        self.config= read_yaml(config_file_path)
        self.params= read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])





    # data ingestion configuration 
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config.data_ingestion

        create_directories([config.data_ingestion_root])

        data_ingestion_config= DataIngestionConfig(
            root_dir= config.data_ingestion_root,
            source_url=config.source_url,
            local_data_file =config.local_data_file,
            unzip_dir =config.unzip_dir,
        )
        return data_ingestion_config


    # prepare model configuration 
    def get_prepare_model_config(self) -> PrepareBaseModelConfig:
        config= self.config.prepare_base_model
        create_directories([config.prepare_base_model_root])

        prepare_base_model_config= PrepareBaseModelConfig(
            root_dir= Path(config.prepare_base_model_root),
            base_model_path= Path(config.base_model_path),
            updated_base_model_path= Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            # params_batch_size=self.params.BATCH_SIZE,
            params_include_top=self.params.INCLUDE_TOP,
            # params_epochs=self.params.EPOCHS,
            params_classes=self.params.CLASSES,
            params_weights=self.params.WEIGHTS,
            params_learning_rate=self.params.LEARNING_RATE, 

        )
        return prepare_base_model_config




    # prepare callbacks config
    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config= self.config.prepare_callbacks
        model_ckpt_dir= os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir),
            ])
        
        prepare_callbacks_config= PrepareCallbacksConfig(
            root_dir= Path(config.prepare_callbacks_root),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir)
        )
        return prepare_callbacks_config
    

    # model trainer config
    def get_training_config(self):
        training= self.config.training
        prepare_base_model= self.config.prepare_base_model
        params= self.params
        training_data= os.path.join(
            self.config.data_ingestion.unzip_dir, 'Chicken-fecal-images'
        )
        create_directories([
            Path(training.root_dir)
        ])

        training_config= TrainingConfig(
            root_dir= Path(training.root_dir),
            trained_model_path= Path(training.trained_model_path),
            training_data= Path(training_data),
            updated_model_path= Path(prepare_base_model.updated_base_model_path),
            params_image_size=params.IMAGE_SIZE,
            params_batch_size=params.BATCH_SIZE,
            params_epochs= params.EPOCHS,
            params_is_augmentation=params.AUGUMENTATION,
        )

        return training_config
    


    # evaluation config
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config= EvaluationConfig(
            path= Path("artifacts/training/model.h5"),
            training_data= Path("artifacts/data_ingestion/Chicken-fecal-images/"),
            all_parameters= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE,
            
        )
        return eval_config
        



In [7]:
from urllib.parse import  urlparse

In [8]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config= config
    

    def _valid_generator(self):
        datagenerator_kwargs= dict(
            rescale= 1/255,
            validation_split= 0.2,
            # batch_size= self.config.batch_size
        )

        dataflow_kwargs= dict(
            target_size= self.config.params_image_size[:-1],
            batch_size= self.config.params_batch_size,
            interpolation= 'bilinear'
        )

        valid_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator= valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset= 'validation',
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path)-> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model= self.load_model(self.config.path)
        self._valid_generator()
        self.score= self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores= {
            'loss': self.score[0],
            'accuracy': self.score[1]
        }
        save_json(path= Path("scores.json"), data=scores)

In [9]:
try:
    config= ConfigurationManager()
    eval_config= config.get_evaluation_config()
    evaluation= Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.save_score()


except Exception as e:
    raise e

Loading configuration from config/config.yaml
Loading parameters from params.yaml
[2023-08-26 18:57:18,660: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-08-26 18:57:18,662: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-26 18:57:18,663: INFO: common: created directory at: artifacts]
Found 78 images belonging to 2 classes.


2023-08-26 18:57:19.464711: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-08-26 18:57:19.740840: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-08-26 18:57:20.321344: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2023-08-26 18:57:20.389045: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2023-08-26 18:57:20.521707: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


3/3 [==============================] - 7s 2s/step - loss: 0.5567 - accuracy: 0.9615
[2023-08-26 18:57:25,756: INFO: common: json file saved at: scores.json]


In [11]:
# EvaluationConfig.path